In [50]:
import findspark
findspark.init()

In [51]:
import pyspark 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [52]:
#Configure Spark
spark = SparkSession\
            .builder\
            .master('local[2]')\
            .appName('quake_etl_process')\
            .config('spark.jars.packages','org.mongodb.spark:mongo-spark-connector_2.12:2.4.1')\
            .getOrCreate()

In [53]:
#Load Data from CSV
df_load = spark.read.csv(r'database.csv', header=True)

#Preview
df_load.take(1)

[Row(Date='01/02/1965', Time='13:44:18', Latitude='19.246', Longitude='145.616', Type='Earthquake', Depth='131.6', Depth Error=None, Depth Seismic Stations=None, Magnitude='6', Magnitude Type='MW', Magnitude Error=None, Magnitude Seismic Stations=None, Azimuthal Gap=None, Horizontal Distance=None, Horizontal Error=None, Root Mean Square=None, ID='ISCGEM860706', Source='ISCGEM', Location Source='ISCGEM', Magnitude Source='ISCGEM', Status='Automatic')]

In [54]:
#Show all Columns
df_load.columns

#List of columns from drop
lst_drop_columns = ['Depth Error','Time','Depth Seismic Stations','Magnitude Error', 'Azimuthal Gap',
'Root Mean Square','Source','Location Source','Magnitude Source','Status','Magnitude Seismic Stations','Horizontal Distance','Horizontal Error']

#Drop columns from list
df_load = df_load.drop(*lst_drop_columns)

#Preview
df_load.show(5)

+----------+--------+---------+----------+-----+---------+--------------+------------+
|      Date|Latitude|Longitude|      Type|Depth|Magnitude|Magnitude Type|          ID|
+----------+--------+---------+----------+-----+---------+--------------+------------+
|01/02/1965|  19.246|  145.616|Earthquake|131.6|        6|            MW|ISCGEM860706|
|01/04/1965|   1.863|  127.352|Earthquake|   80|      5.8|            MW|ISCGEM860737|
|01/05/1965| -20.579| -173.972|Earthquake|   20|      6.2|            MW|ISCGEM860762|
|01/08/1965| -59.076|  -23.557|Earthquake|   15|      5.8|            MW|ISCGEM860856|
|01/09/1965|  11.938|  126.427|Earthquake|   15|      5.8|            MW|ISCGEM860890|
+----------+--------+---------+----------+-----+---------+--------------+------------+
only showing top 5 rows



In [55]:
#Create a Year columns
df_load = df_load.withColumn('Year',year(to_timestamp('Date','dd/MM/yyyy')))

#Preview
df_load.show(5)

+----------+--------+---------+----------+-----+---------+--------------+------------+----+
|      Date|Latitude|Longitude|      Type|Depth|Magnitude|Magnitude Type|          ID|Year|
+----------+--------+---------+----------+-----+---------+--------------+------------+----+
|01/02/1965|  19.246|  145.616|Earthquake|131.6|        6|            MW|ISCGEM860706|1965|
|01/04/1965|   1.863|  127.352|Earthquake|   80|      5.8|            MW|ISCGEM860737|1965|
|01/05/1965| -20.579| -173.972|Earthquake|   20|      6.2|            MW|ISCGEM860762|1965|
|01/08/1965| -59.076|  -23.557|Earthquake|   15|      5.8|            MW|ISCGEM860856|1965|
|01/09/1965|  11.938|  126.427|Earthquake|   15|      5.8|            MW|ISCGEM860890|1965|
+----------+--------+---------+----------+-----+---------+--------------+------------+----+
only showing top 5 rows



In [56]:
#Build the quake freqency DF using the year column and count from each year
df_quake_freq = df_load.groupBy('Year').count().withColumnRenamed('count','Counts')

#Preview
df_quake_freq.show(5)

+----+------+
|Year|Counts|
+----+------+
|1990|   196|
|1975|   150|
|1977|   148|
|2003|   187|
|2007|   211|
+----+------+
only showing top 5 rows



In [57]:
#Print df_load schema
df_load.printSchema()

# cost fileds from string to numeric types
df_load = df_load.withColumn('Latitude',df_load['Latitude'].cast(DoubleType()))\
                .withColumn('Longitude',df_load['Longitude'].cast(DoubleType()))\
                .withColumn('Depth',df_load['Depth'].cast(DoubleType()))\
                .withColumn('Magnitude',df_load['Magnitude'].cast(DoubleType()))

#Print df_load schema after change types
df_load.printSchema()

#Preview
df_load.show(5)


root
 |-- Date: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Depth: string (nullable = true)
 |-- Magnitude: string (nullable = true)
 |-- Magnitude Type: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- Year: integer (nullable = true)

root
 |-- Date: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Type: string (nullable = true)
 |-- Depth: double (nullable = true)
 |-- Magnitude: double (nullable = true)
 |-- Magnitude Type: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- Year: integer (nullable = true)

+----------+--------+---------+----------+-----+---------+--------------+------------+----+
|      Date|Latitude|Longitude|      Type|Depth|Magnitude|Magnitude Type|          ID|Year|
+----------+--------+---------+----------+-----+---------+--------------+------------+----+
|01/02/1965|  19

In [64]:
#Add max and avg Magnitude value for each year
#df_load.groupBy('year').agg(avg("Magnitude").alias("Avg_Magnitude"),max("Magnitude").alias("Max_Magnitude")).show()

df_max = df_load.groupBy('Year').max('Magnitude').withColumnRenamed('max(Magnitude)','Max_Magnitude')
df_max.show(5)

df_avg = df_load.groupBy('Year').avg('Magnitude').withColumnRenamed('avg(Magnitude)','Avg_Magnitude')
df_avg.show(5)

Py4JJavaError: An error occurred while calling o787.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 177.0 failed 1 times, most recent failure: Lost task 0.0 in stage 177.0 (TID 3537, GuyL-NB.call.local, executor driver): java.io.FileNotFoundException: File file:/C:/Users/guyl/Downloads/database.csv does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:124)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:169)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:93)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.sca

In [59]:
#Join max and Avg Dfs to df_quake_freq
df_quake_freq = df_quake_freq.join(df_avg,['Year']).join(df_max,['Year'])

#Preview
df_quake_freq.show(5)

+----+------+-----------------+-------------+
|Year|Counts|    AVG_Magnitude|Max_Magnitude|
+----+------+-----------------+-------------+
|1990|   196|5.858163265306125|          7.6|
|1975|   150| 5.84866666666667|          7.8|
|1977|   148|5.757432432432437|          7.6|
|2003|   187|5.850802139037435|          7.6|
|2007|   211| 5.89099526066351|          8.4|
+----+------+-----------------+-------------+
only showing top 5 rows



In [60]:
#Remove nulls
df_load.dropna()
df_quake_freq.dropna()

DataFrame[Year: int, Counts: bigint, AVG_Magnitude: double, Max_Magnitude: double]

In [61]:
#Preview
df_load.show(5)
df_quake_freq.show(5)

+----------+--------+---------+----------+-----+---------+--------------+------------+----+
|      Date|Latitude|Longitude|      Type|Depth|Magnitude|Magnitude Type|          ID|Year|
+----------+--------+---------+----------+-----+---------+--------------+------------+----+
|01/02/1965|  19.246|  145.616|Earthquake|131.6|      6.0|            MW|ISCGEM860706|1965|
|01/04/1965|   1.863|  127.352|Earthquake| 80.0|      5.8|            MW|ISCGEM860737|1965|
|01/05/1965| -20.579| -173.972|Earthquake| 20.0|      6.2|            MW|ISCGEM860762|1965|
|01/08/1965| -59.076|  -23.557|Earthquake| 15.0|      5.8|            MW|ISCGEM860856|1965|
|01/09/1965|  11.938|  126.427|Earthquake| 15.0|      5.8|            MW|ISCGEM860890|1965|
+----------+--------+---------+----------+-----+---------+--------------+------------+----+
only showing top 5 rows

+----+------+-----------------+-------------+
|Year|Counts|    AVG_Magnitude|Max_Magnitude|
+----+------+-----------------+-------------+
|1990|   

In [62]:
#Builed the collaction in MongoDB
#Write df_load to MongoDB
df_load.write.format('mongo')\
    .mode('overwrite')\
    .option('spark.mongodb.output.uri','mongodb://127.0.0.1:27017/Quake.quakes').save()

In [63]:
#Write df_quake_freq to MongoDB
df_quake_freq.write.format('mongo')\
    .mode('overwrite')\
    .option('spark.mongodb.output.uri','mongodb://127.0.0.1:27017/Quake.quake_freq').save()